In [1]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [2]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __len__(self):
        # return len(self.word2idx)
        return self.idx

In [3]:
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)

        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches * batch_size]
        return ids.reshape(batch_size, -1)

In [4]:
# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
# num_samples = 1000     # number of words to be sampled
num_samples = 100
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [5]:
# Load "Penn Treebank" dataset
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [6]:
# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)
        # Forward propagate LSTM
        out, (h, c) = self.lstm(x, h)
        # Reshape output to (batch_size * sequence_length, hidden_size)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states]

# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i : i + seq_length].to(device)
        targets = ids[:, (i + 1): (i + 1) + seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i + 1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch + 1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

Epoch [1/5], Step[0/1549], Loss: 9.2071, Perplexity: 9968.12
Epoch [1/5], Step[100/1549], Loss: 6.0744, Perplexity: 434.60
Epoch [1/5], Step[200/1549], Loss: 5.9558, Perplexity: 385.98
Epoch [1/5], Step[300/1549], Loss: 5.7529, Perplexity: 315.11
Epoch [1/5], Step[400/1549], Loss: 5.6662, Perplexity: 288.95
Epoch [1/5], Step[500/1549], Loss: 5.1596, Perplexity: 174.10
Epoch [1/5], Step[600/1549], Loss: 5.1753, Perplexity: 176.84
Epoch [1/5], Step[700/1549], Loss: 5.3548, Perplexity: 211.62
Epoch [1/5], Step[800/1549], Loss: 5.1908, Perplexity: 179.61
Epoch [1/5], Step[900/1549], Loss: 5.0640, Perplexity: 158.22
Epoch [1/5], Step[1000/1549], Loss: 5.0752, Perplexity: 160.01
Epoch [1/5], Step[1100/1549], Loss: 5.3590, Perplexity: 212.51
Epoch [1/5], Step[1200/1549], Loss: 5.2103, Perplexity: 183.15
Epoch [1/5], Step[1300/1549], Loss: 5.0607, Perplexity: 157.69
Epoch [1/5], Step[1400/1549], Loss: 4.8641, Perplexity: 129.56
Epoch [1/5], Step[1500/1549], Loss: 5.1716, Perplexity: 176.19
Epo

In [8]:
# Test the model
generated_text = ""
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set intial hidden ane cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # Select one word id randomly
        prob = torch.ones(vocab_size)
        inputs = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(inputs, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            inputs.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)
            generated_text += word

            if (i + 1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i + 1, num_samples, 'sample.txt'))
print(generated_text)

Sampled [100/100] words and save to sample.txt
products late due out to a woman 
mr. freeman is cast distributed to show a <unk> of an aircraft appreciation that is an outside one 
the jewelers team took hits after mr. ingersoll 's won on family patients in N and subsequent exhibition but with the new team 
one thing is very much what since <unk> of oil but he also throws a lot of time including <unk> information about eight years earlier 
the swedish are voluntarily today 
grain elevators built in a program this week about what the government lost several basic monthly trade 


In [9]:
# Save the model checkpoints
torch.save(model.state_dict(), 'model.ckpt')